In [1]:
###import packages
from graph_tool.all import *; import graph_tool.all as gt; 
import  matplotlib; import math; import numpy as np; import pandas as pd;
import pickle; import os; import gc

DEBUG = True

In [2]:
###file locations
path='/media/Data1/tumour_genetics_git/'
g=load_graph(path+"patient_graph_model.xml.gz")
g.set_edge_filter(None)
g.set_vertex_filter(None)

if DEBUG:
    g.set_edge_filter(g.ep.minspanningtree_summed_edges)
    
print("Patient graph loaded")
print(g)

Patient graph loaded
<Graph object, undirected, with 2419 vertices and 2418 edges, 88 internal vertex properties, 58 internal edge properties, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7f5358218dc0, at 0x7f53216374c0>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7f5358218dc0, at 0x7f535aa55a90>, False), at 0x7f5358218dc0>


In [3]:
print("Note we will take the minimum spanning tree of the graph to significnatly reduce the required runtime\nIf running a full analysis (as done in the article) you should use the complete graph (computing power permitting)")

Note we will take the minimum spanning tree of the graph to significnatly reduce the required runtime
If running a full analysis (as done in the article) you should use the complete graph (computing power permitting)


In [4]:
###prepare multigraph weights to pass to stochastic block model
genetic_props_name = []
genetic_props_object = []

for prop in g.ep:
    genetic_props_name.append(prop)
    genetic_props_object.append(g.ep[prop])

temp_prop_dictionary = dict(zip(genetic_props_name,genetic_props_object))

for key in list(temp_prop_dictionary.keys()):
    if "umap" in key:
        del temp_prop_dictionary[key]
    if "summed" in key:
        del temp_prop_dictionary[key]

recs = []
rec_types = []

for i in range(len(list(temp_prop_dictionary.keys()))):
    recs.append(g.ep[list(temp_prop_dictionary.keys())[i]])
    rec_types.append('discrete-binomial')

In [5]:
print("These are the individual edge properties that shall be passed to the graph model")
temp_prop_dictionary

These are the individual edge properties that shall be passed to the graph model


{'19q deletion': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f53216522b0>,
 '1p/19q co-deletion': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f5321652370>,
 '1p/19q deletion': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f5321652430>,
 '1p/19q no co-deletion': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f53216524f0>,
 'ATRX loss': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f53216525b0>,
 'ATRX retained': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f5321652670>,
 'BRAF 1799 T>A': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f5321652730>,
 'BRAF Exon 15-9': <EdgePropertyMap object with value type 'int32_t', for Graph 0x7f5358218dc0, at 0x7f53216527f0>,
 'BRAF Exon 16-11': <EdgePropertyMap object with value type 'int32_t'

In [6]:
print("Running stochastic block models")
state_binomial = gt.minimize_nested_blockmodel_dl(g,state_args=dict(recs=recs,rec_types=rec_types,deg_corr=True))
pickle.dump(state_binomial,open(path+'state_binomial_raw.p','wb'),protocol=4)

state_binomial_ndc = gt.minimize_nested_blockmodel_dl(g,state_args=dict(recs=recs,rec_types=rec_types,deg_corr=False))
pickle.dump(state_binomial_ndc,open(path+'state_binomial_ndc_raw.p','wb'),protocol=4)

Running stochastic block models


In [7]:
print("Comparing initial fits")
N_rows=2; N_cols=7;
entropymodels = pd.DataFrame(np.zeros((N_rows, N_cols)))
entropymodels.columns = ['Model Distribution', 'Fit', 'Nested Block State Entropy','Degree Correction','Model Fit','MCMC Equilibrated Stochastic Block Model Entropy','modelname']
entropymodels.iloc[:,0]="Multivariate Binomial"
entropymodels.iloc[:,1]='Nested'
entropymodels.iloc[0,2]=state_binomial.entropy()
entropymodels.iloc[1,2]=state_binomial_ndc.entropy()
entropymodels.iloc[:,3]='Degree Corrected'
entropymodels.iloc[1,3]='Not Degree Corrected'

entropymodels.iloc[:,4]='Nested, DC'
entropymodels.iloc[1,4]='Nested, NDC'

entropymodels.iloc[0,6]='state_binomial'
entropymodels.iloc[1,6]='state_binomial_ndc'

entropymodels.head()

Comparing initial fits


,Model Distribution,Fit,Nested Block State Entropy,Degree Correction,Model Fit,MCMC Equilibrated Stochastic Block Model Entropy,modelname
0,Multivariate Binomial,Nested,32198.589775,Degree Corrected,"Nested, DC",0.0,state_binomial
1,Multivariate Binomial,Nested,36482.433507,Not Degree Corrected,"Nested, NDC",0.0,state_binomial_ndc


In [8]:
print("Post-SBM optimisation with MCMC")
print("Note we have reduced the number of iterations here for the toy example to reduce runtime")
history  = gt.mcmc_equilibrate(state_binomial, wait=10, nbreaks=2, mcmc_args=dict(niter=1),history=True,verbose=True)
history = pd.DataFrame(history,columns=['nattempts','nmoves','entropy']).astype(float)
history.to_csv(path+'patient_sbm_equilibration.csv')
pickle.dump(state_binomial,open(path+'state_binomial_equilibrated.p','wb'),protocol=4)

Post-SBM optimisation with MCMC
Note we have reduced the number of iterations here for the toy example to reduce runtime
niter:     1  count:    0  breaks:  0  min_S: 32196.670  max_S: 32198.590  S: 32196.670  ΔS:     -1.91984  moves:     6 
niter:     2  count:    0  breaks:  0  min_S: 32192.482  max_S: 32198.590  S: 32192.482  ΔS:     -4.18753  moves:     3 
niter:     3  count:    0  breaks:  0  min_S: 32183.336  max_S: 32198.590  S: 32183.336  ΔS:     -9.14617  moves:    10 
niter:     4  count:    0  breaks:  0  min_S: 32175.187  max_S: 32198.590  S: 32175.187  ΔS:     -8.14948  moves:     3 
niter:     5  count:    1  breaks:  0  min_S: 32175.187  max_S: 32198.590  S: 32185.545  ΔS:      10.3586  moves:     4 
niter:     6  count:    2  breaks:  0  min_S: 32175.187  max_S: 32198.590  S: 32175.923  ΔS:     -9.62283  moves:     4 
niter:     7  count:    3  breaks:  0  min_S: 32175.187  max_S: 32198.590  S: 32178.849  ΔS:      2.92696  moves:   108 
niter:     8  count:    4  break

niter:    68  count:    2  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32041.110  ΔS:     -6.02644  moves:     3 
niter:    69  count:    3  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32042.353  ΔS:      1.24385  moves:    62 
niter:    70  count:    4  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32047.388  ΔS:      5.03449  moves:     1 
niter:    71  count:    5  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32047.388  ΔS:      0.00000  moves:     0 
niter:    72  count:    6  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32044.742  ΔS:     -2.64609  moves:     1 
niter:    73  count:    7  breaks:  1  min_S: 32040.518  max_S: 32185.170  S: 32040.997  ΔS:     -3.74452  moves:     3 
niter:    74  count:    0  breaks:  1  min_S: 32038.167  max_S: 32185.170  S: 32038.167  ΔS:     -2.83014  moves:     3 
niter:    75  count:    0  breaks:  1  min_S: 32032.017  max_S: 32185.170  S: 32032.017  ΔS:     -6.15043  moves:     1 
niter:    76  count:    0  break

In [9]:
print("Complete")

Complete
